In [2]:
import requests
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

## Data Collection

In [120]:
def fetch_all_vacancies(text: str):
    url = "https://api.hh.ru/vacancies"
    params = {
        "text": text,             
        "area": 40,                 
        "per_page": 100,        
        'industry': 7     
    }
    all_vacancies = []
    page = 0 

    while True:
        params["page"] = page
        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            print(f"Ошибка: {response.status_code}, {response.text}")
            break

        data = response.json()
        items = data.get("items", [])
        all_vacancies.extend(items)

        print(f"Обработана страница {page + 1}, найдено вакансий: {len(items)}")
        if page >= data.get("pages", 0) - 1:
            break
        page += 1 

    print(f"Всего собрано вакансий: {len(all_vacancies)}")
    return all_vacancies

In [121]:
data_DA = fetch_all_vacancies("Data analyst")

Обработана страница 1, найдено вакансий: 43
Всего собрано вакансий: 43


In [122]:
data_QA = fetch_all_vacancies("QA Engineer")

Обработана страница 1, найдено вакансий: 91
Всего собрано вакансий: 91


In [123]:
data_SA = fetch_all_vacancies("System analyst")

Обработана страница 1, найдено вакансий: 49
Всего собрано вакансий: 49


In [124]:
data_PM = fetch_all_vacancies("Product Manager")

Обработана страница 1, найдено вакансий: 50
Всего собрано вакансий: 50


In [125]:
data_Back = fetch_all_vacancies('Backend Developer')

Обработана страница 1, найдено вакансий: 100
Обработана страница 2, найдено вакансий: 25
Всего собрано вакансий: 125


In [126]:
data_IOS = fetch_all_vacancies("IOS Developer")

Обработана страница 1, найдено вакансий: 49
Всего собрано вакансий: 49


In [127]:
data_Android = fetch_all_vacancies('Android Developer')

Обработана страница 1, найдено вакансий: 52
Всего собрано вакансий: 52


In [128]:
data_Front = fetch_all_vacancies("Frontend Developer")

Обработана страница 1, найдено вакансий: 100
Обработана страница 2, найдено вакансий: 2
Всего собрано вакансий: 102


In [129]:
data_Devops = fetch_all_vacancies("DevOps")

Обработана страница 1, найдено вакансий: 74
Всего собрано вакансий: 74


In [130]:
data_Security = fetch_all_vacancies('Security engineer')

Обработана страница 1, найдено вакансий: 78
Всего собрано вакансий: 78


In [131]:

def process_vacancies(data):
    vacancies = []
    for item in data:
        salary = item.get("salary")
        vacancies.append({
            "name": item.get("name"),
            "area_name": item.get("area", {}).get("name"),
            "salary_from": salary.get("from") if salary else None,
            "salary_to": salary.get("to") if salary else None,
            "salary_currency": salary.get("currency") if salary else None,
            "published_at": item.get("published_at"),
            "employer_name": item.get("employer", {}).get("name"),
            "requirement": item.get("snippet", {}).get("requirement"),
            "schedule_id": item.get("schedule", {}).get("id"),
            "working_hours_id": item.get("working_hours", [{}])[0].get("id") if item.get("working_hours") else None,
            "work_schedule_by_days": item.get("work_schedule_by_days", [{}])[0].get("name") if item.get("work_schedule_by_days") else None,
            "professional_roles": ", ".join(role.get("name", "") for role in item.get("professional_roles", [])),
            "experience_name": item.get("experience", {}).get("name"),
            "employment_id": item.get("employment", {}).get("id"),
        })
    return pd.DataFrame(vacancies)


In [132]:
df_DA = process_vacancies(data_DA)
df_SA = process_vacancies(data_SA)
df_Backend = process_vacancies(data_Back)
df_QA = process_vacancies(data_QA)
df_IOS = process_vacancies(data_IOS)
df_Android = process_vacancies(data_Android)
df_Security = process_vacancies(data_Security)
df_Front = process_vacancies(data_Front)
df_DevOps = process_vacancies(data_Devops)
df_PM = process_vacancies(data_PM)

In [142]:
df_full = [df_Front,df_DevOps,df_Android,df_Backend,df_DA,df_IOS,df_PM,df_SA,df_Security,df_QA]
df_full = pd.concat(df_full,ignore_index=True)

In [143]:
len(df_full)

713

In [144]:
df_full.to_csv("data/all_vacancies.csv", index=False)
df_Android.to_csv('data/android_vacancies.csv',index=False)
df_Backend.to_csv('data/backend_vacancies.csv', index=False)
df_PM.to_csv('data/pm_vacancies.csv', index=False)
df_DA.to_csv('data/da_vacancies.csv', index=False)
df_QA.to_csv('data/qa_vacancies.csv', index=False)
df_IOS.to_csv('data/ios_vacancies.csv', index=False)
df_DevOps.to_csv('data/devops_vacancies.csv', index=False)
df_Security.to_csv('data/security_vacancies.csv', index=False)
df_Front.to_csv('data/frontend_vacancies.csv', index=False)
df_SA.to_csv("data/sa_vacancies.csv", index=False)

## Data Analysis

In [1]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
import string
from sklearn.feature_extraction.text import CountVectorizer


In [3]:
nltk.download('stopwords')
stop_words = set(stopwords.words('russian') + stopwords.words('english'))
custom_sw = ['опыт','работы',"experience",'знание','понимание','знания','умение','knowledge',
             'принципов','working','highlighttextdatahighlighttext','ability','–','—',
             'highlighttext','лет','years','навыки','работать','understanding','инструментами']
stop_words.update(custom_sw)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dias\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def get_req_freqeuncy(data: list):
    text = " ".join(data)
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    words = [word for word in text.split() if word not in stop_words]

    return Counter(words)

def get_req_phrases(data:list):
    vectorizer = CountVectorizer(ngram_range=(2, 3), stop_words=list(stop_words))
    X = vectorizer.fit_transform(data)
    phrases = vectorizer.get_feature_names_out()
    counts = X.toarray().sum(axis=0)
    return Counter(dict(zip(phrases, counts)))



In [5]:
df_DA = pd.read_csv("data/da_vacancies.csv")
df_Back = pd.read_csv('data/backend_vacancies.csv')
df_PM = pd.read_csv('data/pm_vacancies.csv')

In [6]:
req_DA = get_req_freqeuncy(df_DA['requirement'].to_list())

In [7]:
print("Самые часто встречающиеся слова в требованиях Дата Аналитики")
req_DA.most_common(15)

Самые часто встречающиеся слова в требованиях Дата Аналитики


[('данных', 8),
 ('2', 8),
 ('highlighttextаналитикиhighlighttext', 4),
 ('python', 4),
 ('science', 4),
 ('integration', 4),
 ('sql', 4),
 ('архитектуры', 4),
 ('dbt', 3),
 ('системами', 3),
 ('analytics', 3),
 ('excel', 3),
 ('power', 3),
 ('highlighttextanalysthighlighttext', 3),
 ('big', 3)]

In [8]:
phrases_DA = get_req_phrases(df_DA['requirement'].to_list())

In [9]:
print("Самые часто встречающиеся фразы в требованиях Дата Аналитики")
phrases_DA.most_common(15)

Самые часто встречающиеся фразы в требованиях Дата Аналитики


[('data integration', np.int64(4)),
 ('big data', np.int64(3)),
 ('data science', np.int64(3)),
 ('data visualization', np.int64(3)),
 ('advanced excel', np.int64(2)),
 ('advanced excel google', np.int64(2)),
 ('airflow jupyter', np.int64(2)),
 ('airflow jupyter notebook', np.int64(2)),
 ('analytics data', np.int64(2)),
 ('analytics data modelling', np.int64(2)),
 ('arrays pivots', np.int64(2)),
 ('arrays pivots data', np.int64(2)),
 ('banking data', np.int64(2)),
 ('banking data data', np.int64(2)),
 ('build tool', np.int64(2))]